In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import *

import scipy.stats as stats
import statsmodels.stats.weightstats as wstats
%matplotlib inline

In [ ]:
!cat experiment_stats.csv | head -3

In [ ]:
data = pd.read_csv('experiment_stats.csv')
data.head()

In [ ]:
data.mean()

In [ ]:
plt.figure(figsize=(15,8))
data.query('70000 > ord_value > 0').ord_value.hist(bins=200)
#data.query('variant == 0 & 50000 > ord_value > 0').ord_value.hist(bins=100)

In [ ]:
# Для удобства добавим колонку "конверсия"
data['conversion'] = data['ord_value'].apply(lambda x: 1 if x > 0 else 0)

data.groupby('variant').mean()

In [ ]:
# CR MONITORING BY VARIATIONS

# We can consider row number as time-like value


# Calculating cumulative CR
c0 = 100*data[data['variant'] == 0].expanding(1500).mean()['conversion'][:800000]
c1 = 100*data[data['variant'] == 1].expanding(1500).mean()['conversion'][:800000]

# create dashboard
fig, ax = plt.subplots(figsize=(14,6))    
plot(c0)
plot(c1)
title('Cumulative conversion rate', fontdict={'size':16})

xlabel('sessions')
ylabel('conversion')
plt.grid(True)
xlim(0, 1500000)
ylim(3.5,5)

## Two Samples z-test for Proportions

## $z = \frac{\hat{p_1}-\hat{p_2}}{\sqrt{\hat{p} (1-\hat{p}) (\frac{1}{n_1} + \frac{1}{n_2})}} $
where

### $\hat{p_1} = \frac{x_1}{n_1}, \hat{p_2} = \frac{x_2}{n_2} $
### $\hat{p} = \frac{x_1 + x_2}{n_1 + n_2}$
$x_1, x_2$ - number of successes in group 1 and 2

$n_1, n_2$ - number of observations in group 1 and 2

In [ ]:
# implementation from scratch
def ztest_proportion_two_samples(x1, n1, x2, n2, one_sided=True, verbose=False):
    p1 = x1/n1
    p2 = x2/n2    

    p = (x1+x2)/(n1+n2)
    denom = sqrt(p*(1-p)*(1/n1+1/n2))
    
    z = (p1-p2)/denom
    p = 1-stats.norm.cdf(abs(z))
    p *= 2-one_sided # if not one_sided: p *= 2
    
    if verbose:
        print(xa, na, xb, nb)
        print('z-stat = {z}'.format(z=z))
        print('p-value = {p}'.format(p=p))

    return p

In [ ]:
def CR_stat_test(data, k, verbose=False):
    d = data.iloc[:k, :]

    xa = d[d.variant == 0].conversion.sum()
    na = d[d.variant == 0].conversion.count()

    xb = d[d.variant == 1].conversion.sum()
    nb = d[d.variant == 1].conversion.count()

    p = ztest_proportion_two_samples(xa, na, xb, nb, one_sided=False, verbose=verbose)
    
    return p, k

In [ ]:
CR_stat_test(data, 10000, verbose=True)

In [ ]:
pv = [CR_stat_test(data, 10000*i, verbose=False) for i in range(1,150)]

In [ ]:
# P-value versus sample size
fig, ax = plt.subplots(figsize=(14,6))    
plot([dot[1] for dot in pv], [dot[0] for dot in pv])
plot([0, 2000000], [0.05, 0.05], color='red', linestyle='dashed')


title('P-Value', fontdict={'size':16})

xlabel('split size')
plt.grid(True)
xlim(0, 1400000)
ylim(0,1)

In [ ]:
# using statsmodels
from statsmodels.stats.proportion import proportions_ztest

count = np.array([xa, xb])
nobs = np.array([na, nb])
z,p = proportions_ztest(count, nobs, value=0, alternative='two-sided')
print(' z-stat = {z} \n p-value = {p}'.format(z=z,p=p))

In [ ]:
data.sort_values(by='ord_value',ascending=False).head()

In [ ]:
from scipy.stats import mannwhitneyu
print(mannwhitneyu.__doc__)

In [ ]:
#d = data

k = 120000
d = data.iloc[:k, :]

test = d[d.variant == 1].ord_value
control = d[d.variant == 0].ord_value

mannwhitneyu(test, control, alternative='greater')[1]